従来法のMMR-Wをjuliaにて実装  
提案手法の実装が正しいことを確認するために、先行研究の結果と比較する

In [1]:
# import Pkg
# Pkg.add("DataFrames")
# Pkg.add("CSV")
# Pkg.add("IntervalSets")
# Pkg.add("LatexStrings")

In [2]:
using DataFrames, CSV
using IntervalArithmetic
using LaTeXStrings
using Statistics
# using Base

include("./libs/display-latex.jl")
include("./libs/crisp-pcm.jl")
include("./libs/solve-deterministic-ahp.jl")
include("./libs/analysis-indicators.jl")
include("./libs/MMR-W.jl")

MMR_W (generic function with 1 method)

In [3]:
# 評価基準数
N = 6;
# 真の区間重要度のセッティング
true_importance = "B"; # ["A", "B", "C", "D", "E"]

In [4]:
# 真の区間重要度を読み込み
df = CSV.File("./data/N"*string(N)*"_"*true_importance*"_Given_interval_weight.csv", header=false) |> DataFrame
T = Vector{Interval{Float64}}() # Float64のところにTを使うとおかしくなるのはなぜか？
for i in 1:N
    push!(T, interval(df[1, 2*i-1], df[1, 2*i]))
end

display(L"T = %$(intervalVectorLaTeXString(T))")

L"$T = \begin{pmatrix} \left[ 0.16, 0.3 \right] \\ \left[ 0.14, 0.26 \right] \\ \left[ 0.13, 0.23 \right] \\ \left[ 0.11, 0.19 \right] \\ \left[ 0.1, 0.16 \right] \\ \left[ 0.09, 0.13 \right] \end{pmatrix}$"

In [5]:
# 一対比較行列を1000個読み込み
df1 = CSV.File("./data/N"*string(N)*"_a3_"*true_importance*"_PCM_int.csv", header=false) |> DataFrame

# データフレームをN×Nのサブデータフレームに分割
subdfs = split_dataframe(df1, N);

In [ ]:
# 結果格納用
P = Matrix{Float64}(undef, N, 1000)
Q = Matrix{Float64}(undef, N, 1000)
R = Matrix{Float64}(undef, N, 1000)
F = Matrix{Float64}(undef, N, 1000)

# 1000個のデータで推定
for (i, subdf) in enumerate(subdfs)

    # 一対比較行列を取り出す
    A = Matrix(subdf)
    # display(L"A = %$(matrixLaTeXString(A))")

    # 提案手法で区間重要度推定
    solution = MMR_W(A, EV)
    E = solution.W # 区間重要度の推定値
    # display(L"E = %$(intervalVectorLaTeXString(E))")
    # display(L"T = %$(intervalVectorLaTeXString(T))")

    # for j in 1:N
    #     if c_diam(E[j]) == 0
    #         println(i, j)
    #     end
    # end

    TcapE = T .∩ E # 真の区間重要度と推定値の積集合
    TcupE = T .∪ E # 真の区間重要度と推定値の和集合
    # display(L"T ∩ E = %$(intervalVectorLaTeXString(TcapE))")
    # display(L"T ∪ E = %$(intervalVectorLaTeXString(TcupE))")
    
    # P, Q, R, F の計算
    P[:, i] = calculate_P(T, E) # 真の区間重要度と推定値の積集合の割合
    Q[:, i] = calculate_Q(T, E) # 真の区間重要度と推定値の積集合の割合
    R[:, i] = calculate_R(T, E) # 真の区間重要度と推定値の積集合の割合
    F[:, i] = calculate_F(T, E) # F値
    # println(i)
    # display(L"P = %$(VectorLaTeXString(P[:,i]))")
    # display(L"Q = %$(VectorLaTeXString(Q[:,i]))")
    # display(L"R = %$(VectorLaTeXString(R[:,i]))")
    # display(L"F = %$(VectorLaTeXString(F[:,i]))")
end

In [7]:
P_ave = mean(P)
Q_ave = mean(Q)
R_ave = mean(R)
F_ave = mean(F)
display(L"Pₐᵥₑ = %$(round(P_ave, digits=5))")
display(L"Qₐᵥₑ = %$(round(Q_ave, digits=5))")
display(L"Rₐᵥₑ = %$(round(R_ave, digits=5))")
display(L"Fₐᵥₑ = %$(round(F_ave, digits=5))")

L"$Pₐᵥₑ = 0.51973$"

L"$Qₐᵥₑ = 0.76483$"

L"$Rₐᵥₑ = 0.68693$"

L"$Fₐᵥₑ = 0.65495$"

In [8]:
P_var = var(P)
Q_var = var(Q)
R_var = var(R)
F_var = var(F)
display(L"Pᵥₐᵣ = %$(round(P_var, digits=5))")
display(L"Qᵥₐᵣ = %$(round(Q_var, digits=5))")
display(L"Rᵥₐᵣ = %$(round(R_var, digits=5))")
display(L"Fᵥₐᵣ = %$(round(F_var, digits=5))")

L"$Pᵥₐᵣ = 0.0456$"

L"$Qᵥₐᵣ = 0.08564$"

L"$Rᵥₐᵣ = 0.05437$"

L"$Fᵥₐᵣ = 0.0444$"